In [2]:
import xml.dom.minidom
import xml.sax
import pandas as pd

In [23]:


def get_poems():
    table = pd.read_excel('poem.xlsx')
    word_list_df = pd.read_excel('wordlist.xlsx')
    
    authors_name=set()
    poems_all=[]
    temp=[]
    temp_id=-1
    for i in range(len(table)):
        x=table.iloc[i]
        poem_id = x.Poem_id
        line_number=x.line_number
        
        if poem_id!=temp_id :
            if temp_id!=-1 and len(temp)>=4:
                poems_all.append(tuple(temp))
            temp_id=poem_id
            temp=[]

        contance = x.content
        if line_number==-1:
            authors_name.add(contance[2:])

        if line_number==-1 or line_number==-100:
            temp.append(contance[2:])
        elif line_number==-10:
            pass
        else:
            temp.append(contance)
    
    dic=dict()
    for au in authors_name:
        dic[au]=[]
    for poem in poems_all:
        dic[poem[1]].append(poem)
    return dic

In [3]:
def get_id2poem():
    table = pd.read_excel('poem.xlsx')
    word_list_df = pd.read_excel('wordlist.xlsx')

    authors_name=set()
    poems_all=dict()
    temp=[]
    temp_id=-1
    for i in range(len(table)):
        x=table.iloc[i]
        poem_id = x.Poem_id
        line_number=x.line_number
        
        if poem_id!=temp_id :
            if temp_id!=-1 and len(temp)>=4:
                poems_all[temp_id]=tuple(temp)
            temp_id=poem_id
            temp=[]

        contance = x.content
        if line_number==-1:
            authors_name.add(contance[2:])

        if line_number==-1 or line_number==-100:
            temp.append(contance[2:])
        elif line_number==-10:
            pass
        else:
            temp.append(contance)


    return poems_all

In [4]:
# id2poem=get_id2poem()

In [7]:
# import numpy as np
# np.save('id2poem.npy',id2poem)

In [9]:
# id2poem[384]

KeyError: 384

### 发布者-订阅者模式

实现 设计模式中的观察者模式，本部分将以第一部分中提取的诗歌信息作为主题。

观察者模式也被称为发布-订阅（Publish/Subscribe）模式。当这个主题对象状态变化时，会通知所有观察者对象并作出相应处理逻辑。

在本问题中，可以形象地理解成，Publisher为诗歌发布中心，Reader为订阅相应诗人的诗歌的人。需要实现订阅，退订，Publisher可以发布新增、删除诗歌的消息以及当前各位诗人的诗歌的总数。当Publisher发布消息时，它需要把需要推送给所有订阅的Reader。


**Publisher（发布者）**

    - 保存订阅的读者
    - 记录四位诗人各自诗歌的总数
    - 发布四位诗人新诗的消息，每次只需要发布一首
    - 发布新消息时，发布给所有已订阅的读者   
   
**Reader（订阅者）**

    - 订阅/取消订阅 发布者
    - 保存发布者发来的诗歌
    - 保存所有消息，这意味着这类读者需要将四位诗人的诗歌分类。
 


#### 参考资料

[1. Observer pattern Wiki](https://en.wikipedia.org/wiki/Observer_pattern)   
[2. 设计模式之观察者模式（c++）](https://www.cnblogs.com/carsonzhu/p/5770253.html)

In [22]:
import threading

In [23]:
class BasePublisher(object):
    def __init__(self,name):
        self.authors = list(get_poems().keys())
        self.name=name
        self.totalNumber=dict()
        self.readers = dict()
        
        for au in self.authors:
            self.totalNumber[au]=0
            self.readers[au]=[]
            
        self.poems = []
        
    def subscribeReader(self, author, reader):
        self.readers[author].append(reader)
        return self

    def unsubscribeReader(self, author, reader):
        self.readers[author].remove(reader)
        return self

    def notifyReader(self,author, poem):
        pass
    
    def __str__(self):
        readers = f'Subscribed Reader: {set([reader.name for author in self.authors for reader in self.readers[author]])}\n'
        poems = f'发出的诗歌:{[(key,value) for key,value in self.totalNumber.items()]}\n'
        return readers + poems

class Publisher(BasePublisher):
    def __init__(self,name):
        BasePublisher.__init__(self,name)
        
    def notifyReader(self, author, poem):
        self.totalNumber[author] += 1
        news = '{}的诗{}'.format(author,poem)
        self.poems.append(news)
        for reader in self.readers[author]:
            reader.receivePoem(self, poem, author)

In [24]:
class BaseReader(object):

    def __init__(self):
        # BaseReader 的初始化方法
        # TODO
        self.name=""
        self.publishers=[]
        self.authors = []
        
    def subscribeToPublisher(self, author, publisher):
        # Reader向Publisher订阅
        # TODO
        publisher.subscribeReader(author, self)
        self.publishers.append(publisher)
        self.authors.append(author)
        
    def unsubscribeToPublisher(self, author, publisher):
        # Reader向Publisher取消订阅
        # TODO
        publisher.unsubscribeReader(author, self)
        self.authors.remove(author)
        
    def __str__(self):
        pubs = f'Subscribed Publisher: {[publisher.name for publisher in self.publishers]}\n'
        return pubs
    
    def receivePoem(self, publisher, poem , author):
        pass
    
    def printStatistics(self):
        # 打印消息
        pass

In [25]:
class Reader(BaseReader):

    def __init__(self, _name):
        BaseReader.__init__(self)
        self.name=_name
        self.poems =  {"李白":[],"杜甫":[],"白居易":[],"王維":[]}
        
    def receivePoem(self, publisher, poem, author):
        if poem not in self.poems[author]:
            self.poems[author].append(poem)
        
    def printStatistics(self):
        print("{}存有".format(self.name))
        for key in self.poems.keys():
            if (len(self.poems[key])>0):
                print(f'诗人{key}的诗有:')
                for poem in self.poems[key]:
                    print(poem)

In [26]:
if __name__ == "__main__":
    poem_dist = get_poems()
    
    publisher = Publisher("Publisher")
    Alice=Reader('Alice')
    Bob=Reader('Bob')
    
    Alice.subscribeToPublisher("李白", publisher)
    Bob.subscribeToPublisher("杜甫",publisher)
    
    publisher.notifyReader("李白",poem_dist["李白"][0])
    
    Alice.printStatistics()
    Bob.printStatistics()
    
    publisher.notifyReader("杜甫",poem_dist["杜甫"][1])
   
    Alice.printStatistics()
    Bob.printStatistics()
    
    Bob.unsubscribeToPublisher("杜甫", publisher)
    
    publisher.notifyReader("白居易",poem_dist["白居易"][2])
    print(publisher)

Alice存有
诗人李白的诗有:
('登峴山亭寄晉陵張少府', '李白', '峴首風湍急', '雲帆若鳥飛', '憑軒試一問', '張翰欲來歸')
Bob存有
Alice存有
诗人李白的诗有:
('登峴山亭寄晉陵張少府', '李白', '峴首風湍急', '雲帆若鳥飛', '憑軒試一問', '張翰欲來歸')
Bob存有
诗人杜甫的诗有:
('因崔五侍御寄高彭州  適', '杜甫', '百年已過半', '秋至轉饑寒', '為問彭州牧', '何時救急難')
Subscribed Reader: {'Alice'}
发出的诗歌:[('李白', 1), ('杜甫', 1), ('白居易', 1), ('王維', 0)]

